In [1]:
import os.path as osp
from pathlib import Path
import torch
from torch.nn import Linear
import torch.nn.functional as F

import torch_geometric.transforms as T
from dataset import MovieLens
from torch_geometric.nn import SAGEConv, to_hetero

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

path = osp.join(Path().resolve(), 'data')
dataset = MovieLens(path, model_name='all-MiniLM-L6-v2')
data = dataset[0].to(device)

# Add user node features for message passing:
data['user'].x = torch.eye(data['user'].num_nodes, device=device)
del data['user'].num_nodes

# Add a reverse ('movie', 'rev_rates', 'user') relation for message passing:
data = T.ToUndirected()(data)
del data['movie', 'rev_rates', 'user'].edge_label 

In [3]:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('user', 'rates', 'movie')],
    rev_edge_types=[('movie', 'rev_rates', 'user')],
)(data)

# We have an unbalanced dataset with many labels for rating 3 and 4, and very few for 0 and 1. Therefore we use a weighted MSE loss.
# Count the frequency of each value in an array of non-negative ints: https://pytorch.org/docs/stable/generated/torch.bincount.html
weight = torch.bincount(train_data['user', 'movie'].edge_label)
# Take the maximum number of appearences of a rate and normalize all with that number
weight = weight.max() / weight

In [4]:
# Model from: https://github.com/vicbeldo98/GNN_mini_example/blob/main/train.py
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)


In [5]:
def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()

model = Model(hidden_channels=32).to(device)

# Due to lazy initialization, we need to run one model step so the number
# of parameters can be inferred:
with torch.no_grad():
    model.encoder(train_data.x_dict, train_data.edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [6]:
def train():
    model.train()
    optimizer.zero_grad()

    pred = model(train_data.x_dict, train_data.edge_index_dict, train_data['user', 'movie'].edge_label_index)
    target = train_data['user', 'movie'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)

In [7]:
@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict, data['user', 'movie'].edge_label_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'movie'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)

In [8]:
for epoch in range(1, 51):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, 'f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')
    if epoch%10==0:
        torch.save(model.state_dict(),f"models/model{epoch//10}.pt")

Epoch: 001, Loss: 19.3783, Train: 3.3489, Val: 3.3595, Test: 3.3563
Epoch: 002, Loss: 17.4074, Train: 3.0623, Val: 3.0742, Test: 3.0706
Epoch: 003, Loss: 14.6315, Train: 2.5345, Val: 2.5482, Test: 2.5439
Epoch: 004, Loss: 10.4649, Train: 1.6693, Val: 1.6817, Test: 1.6765
Epoch: 005, Loss: 6.4561, Train: 1.1545, Val: 1.1255, Test: 1.1263
Epoch: 006, Loss: 9.1111, Train: 1.1160, Val: 1.0917, Test: 1.0915
Epoch: 007, Loss: 8.4446, Train: 1.1875, Val: 1.1876, Test: 1.1832
Epoch: 008, Loss: 6.2441, Train: 1.5928, Val: 1.6037, Test: 1.5986
Epoch: 009, Loss: 6.2099, Train: 1.9043, Val: 1.9169, Test: 1.9121
Epoch: 010, Loss: 7.0958, Train: 2.0015, Val: 2.0143, Test: 2.0096
Epoch: 011, Loss: 7.4739, Train: 1.9281, Val: 1.9409, Test: 1.9360
Epoch: 012, Loss: 7.1557, Train: 1.7335, Val: 1.7458, Test: 1.7407
Epoch: 013, Loss: 6.4649, Train: 1.4708, Val: 1.4811, Test: 1.4757
Epoch: 014, Loss: 5.8725, Train: 1.2261, Val: 1.2312, Test: 1.2259
Epoch: 015, Loss: 5.8027, Train: 1.0963, Val: 1.0946, Test